In [1]:
!wget http://nlp.cs.washington.edu/pair2vec/spanbert_large.tar.gz

--2020-01-20 17:32:32--  http://nlp.cs.washington.edu/pair2vec/spanbert_large.tar.gz
Resolving nlp.cs.washington.edu (nlp.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to nlp.cs.washington.edu (nlp.cs.washington.edu)|128.208.3.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4340337513 (4.0G) [application/x-gzip]
Saving to: ‘spanbert_large.tar.gz’

spanbert_large.tar. 100%[===================>]   4.04G   151KB/s    in 8h 13m  

2020-01-21 01:45:39 (143 KB/s) - ‘spanbert_large.tar.gz’ saved [4340337513/4340337513]



# Mount/Import

In [0]:
from google.colab import auth
auth.authenticate_user()
project_id = 'topicosmultimedia'
bucket_name = 'loliveira_test_bucket'
!gcloud config set project {project_id}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [0]:
from google.colab import drive
import sys
drive.mount('/content/gdrive/',force_remount=True)

root = r"/content/gdrive/My Drive/Puc/Projeto Final"
#root = r"/content/gdrive/My Drive/ProjetoFinal"
sys.path.append(f'{root}/Code/source')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import os

if not os.path.isdir("data"):
  print("No data folder. Building all")
  #!pip uninstall tensorflow -y
  #!pip install tensorflow==2.0.0
  #!pip install tensorflow==1.13.1
  #!7za e '{root}/Datasets/mention_training/full_diag_40.7z' -odata -mmt4
  #!7za x '{root}/train_sliding.7z' -odata -mmt4 -y
  !7za x '{root}/Datasets/tuned/train_sliding.7z' -odata -mmt4 -y
  #!mkdir data

No data folder. Building all

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/My Drive/Puc/Projeto Final/Datasets/tuned/                                                                    1 file, 6982455881 bytes (6659 MiB)

Extracting archive: /content/gdrive/My Drive/Puc/Projeto Final/Datasets/tuned/train_sliding.7z
--
Path = /content/gdrive/My Drive/Puc/Projeto Final/Datasets/tuned/train_sliding.7z
Type = 7z
Physical Size = 6982455881
Headers Size = 81576
Method = LZMA2:24
Solid = +
Blocks = 10

  0%      0% 225 - sliding/bc/cctv/00/cctv_0001.sliding                                      

In [0]:
#from __future__ import absolute_import, division, print_function, unicode_literals
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
import numpy as np

## Model

In [0]:
BUFFER_SIZE = 5000
EPOCHS = 100
VALIDATION_STEPS = 8
VALIDATIONS_PER_EPOCH = 5
BATCH_SIZE = 200
REDUCTIONS_STEPS=5

#DATA_FOLDER="data"
#DATA_FOLDER=f"gs://loliveira_tsv_repo/mention_training/multiple_{MAX_NUM_MENTIONS}"
#DATA_FOLDER=f"/content/gdrive/My Drive/ProjetoFinal/tsv/diagonal_{MAX_NUM_MENTIONS}"
#DATA_FOLDER=f"/content/gdrive/My Drive/ProjetoFinal/tsv/full_diag_{MAX_NUM_MENTIONS}"
#DATA_FOLDER=f"/content/gdrive/My Drive/ProjetoFinal/tsv/cumsum_{MAX_NUM_MENTIONS}"
#DATA_FOLDER=f"{root}/tsv/separation_40"
DATA_FOLDER=f"{root}/Datasets/separation_40"
os.environ["BERT_DB_PATH"]=f"data/sliding"

## Running

histories = cross_validate(DATA_FOLDER)

for stat in ['val_recall', 'val_precision', 'val_f1_score']:
  val_recall = np.mean([h.history[stat][-1] for h in histories])
  std_recall = np.std([h.history[stat][-1] for h in histories])
  print(f"Mean {stat}:{val_recall} +- {std_recall}")

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from imp import reload
from training import train_mentions, train_mentions_model
from extension import bert_features

reload(train_mentions)
reload(train_mentions_model)
reload(bert_features)

callbacks=[
  tf.keras.callbacks.ModelCheckpoint('model.h5', verbose = 1, save_best_only = True ),
  tf.keras.callbacks.CSVLogger("log2.csv")
]

history, encoder, model = train_mentions.train_eval(DATA_FOLDER , callbacks, True, \
                                                    batch_size=BATCH_SIZE, \
                                                    buffer_size=BUFFER_SIZE,\
                                                    validation_steps=VALIDATION_STEPS, \
                                                    steps_per_epoch=(VALIDATION_STEPS * VALIDATIONS_PER_EPOCH),\
                                                    reduction_steps=REDUCTIONS_STEPS)

encoder.save_weights(f"encoder.h5")
!gsutil -m cp *.h5 gs://{bucket_name}/cort/models/arc_embedding/

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Emb_BatchNorm_1.shape: (?, 780, 1, 128)
cumsum:(?, 780, 3, 768)
Cumsum_Permute_1:(?, 780, 768, 3)
Cumsum_Conv_1:(?, 780, 1, 256)
Cumsum_Permute_2:(?, 780, 256, 1)
Cumsum_Conv_2:(?, 780, 1, 128)
Final_Encoder.shape: (?, 780, 2, 128)
Final_Conv_1.shape: (?, 780, 1, 128)
Final_BatchNorm.shape: (?, 780, 128)
Decoder.shape: (?, 780, 2)
Model: "complete"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_embeddings (InputLayer)   [(None, 40, 768)]    0                                            
__________________________________________________________________________________________________
Emb_Encoder (Reshape)           (None, 40, 768, 1)   0           input_embeddings[0][0]           
_________________________________________________________________________

In [0]:
sequence={"separador":[]}
for k in list(history[0].history.keys()):
  sequence[k] = []  

for h in history:
  for chart, values in h.history.items():
    sequence[chart] += values
  sequence["separador"].append(len(values))

In [0]:
import matplotlib.pyplot as plt

#history = model.fit(x, y, validation_split=0.25, epochs=50, batch_size=16, verbose=1)

# Plot training & validation accuracy values
plt.plot(sequence['recall'])
plt.plot(sequence['val_recall'])
plt.title('Model Recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(sequence['loss'])
plt.plot(sequence['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(sequence['f1_score'])
plt.plot(sequence['val_f1_score'])
plt.title('F1')
plt.ylabel('F1')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(sequence['precision'])
plt.plot(sequence['val_precision'])
plt.title('Model Precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Testando Predict

In [0]:
files = get_files(DATA_FOLDER)
data_fn = input_fn_builder(files[0], MAX_NUM_MENTIONS, MAX_NUM_PREDICTIONS, False)
predict_data = data_fn({"batch_size": BATCH_SIZE})
x = model.predict(predict_data, steps=10)

In [0]:
with tf.Session():
  x = predict_data.make_one_shot_iterator().get_next()
  x[0]["input_embeddings"].eval()